# 01 Import libraries 

In [2]:
import os, re, math, textwrap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# Display settings
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 200)

# 02 Import Data

In [4]:
path = r'/Users/woodoooo/Desktop/World University Rankings'

In [5]:
file2026 = os.path.join (path, '02_Data', 'Original Data', '2026 QS World University Rankings.xlsx')

In [6]:
df_uni26 = pd.read_excel(file2026)

In [7]:
for col in df_uni26.columns.tolist():
      weird = (df_uni26[[col]].map(type) != df_uni26[[col]].iloc[0].apply(type)).any(axis = 1)
      if len (df_uni26[weird]) > 0:
        print (col)
        

Rank
Previous Rank
Size
Research
Status
AR RANK
ER RANK
FSR RANK
CPF RANK
IFR RANK
ISR RANK
ISD RANK
IRN RANK
EO RANK
SUS RANK
Overall SCORE


In [8]:
df_uni26.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1503 entries, 0 to 1502
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Index              1503 non-null   int64  
 1   Rank               1503 non-null   object 
 2   Previous Rank      1390 non-null   object 
 3   Name               1503 non-null   object 
 4   Country/Territory  1503 non-null   object 
 5   Region             1503 non-null   object 
 6   Size               1502 non-null   object 
 7   Focus              1503 non-null   object 
 8   Research           1502 non-null   object 
 9   Status             1455 non-null   object 
 10  AR SCORE           1503 non-null   float64
 11  AR RANK            1503 non-null   object 
 12  ER SCORE           1503 non-null   float64
 13  ER RANK            1503 non-null   object 
 14  FSR SCORE          1503 non-null   float64
 15  FSR RANK           1503 non-null   object 
 16  CPF SCORE          1503 

In [9]:
df_uni26.shape

(1503, 31)

In [10]:
df_uni26.columns

Index(['Index', 'Rank', 'Previous Rank', 'Name', 'Country/Territory', 'Region', 'Size', 'Focus', 'Research', 'Status', 'AR SCORE', 'AR RANK', 'ER SCORE', 'ER RANK', 'FSR SCORE', 'FSR RANK',
       'CPF SCORE', 'CPF RANK', 'IFR SCORE', 'IFR RANK', 'ISR SCORE', 'ISR RANK', 'ISD SCORE', 'ISD RANK', 'IRN SCORE', 'IRN RANK', 'EO SCORE', 'EO RANK', 'SUS SCORE', 'SUS RANK', 'Overall SCORE'],
      dtype='object')

# 03 Missing Values: Check & Handle

In [11]:
df_uni26[df_uni26['Previous Rank'].isna()][['Name', 'Previous Rank']]

,Name,Previous Rank
309,"City St George’s, University of London",NaN
361,University of Chinese Academy of Sciences (UCAS),NaN
508,China Medical University,NaN
530,The Education University of Hong Kong,NaN
638,"Lincoln University College, Malaysia",NaN
...,...,...
1490,University of Da Nang,NaN
1491,University of Hawaii at Hilo,NaN
1493,University of Moratuwa,NaN
1497,University of Tripoli,NaN


In [12]:
df_uni26[df_uni26['Size'].isna()][['Name', 'Size', 'Country/Territory', 'Region']]

,Name,Size,Country/Territory,Region
1242,Islamic Azad University,NaN,Iran (Islamic Republic of),Asia


In [13]:
# This NaN ('Size') should be replaced with XL, based on the QS methodology and information found.

# Islamic Azad University is a massive institution, with 10 million square meters of university building space 
# on 35 million square meters of property, and an estimated $20-25 billion in assets, 
# making it one of the largest universities in the world by physical footprint and assets. 

In [14]:
df_uni26.loc[df_uni26['Name'] == 'Islamic Azad University', 'Size'] = 'XL'

In [15]:
df_uni26[df_uni26['Research'].isna()][['Name', 'Research', 'Country/Territory', 'Region']]

,Name,Research,Country/Territory,Region
1242,Islamic Azad University,NaN,Iran (Islamic Republic of),Asia


In [16]:
# This NaN ('Research') should be replaced with VH, based on the QS methodology and information found.

# For Islamic Azad University (IAU): It is one of the world’s most prolific universities 
# in terms of Scopus publications. In fact, IAU consistently appears near the top worldwide for number of publications (because it has hundreds of campuses and a very large academic staff).
# This scale of output places it in the highest category of research activity. 
# The correct value for Research Intensity for Islamic Azad University should be replaced with Very High value of research intencity

In [17]:
df_uni26.loc[df_uni26['Name'] == 'Islamic Azad University', 'Research'] = 'VH'

In [18]:
df_uni26[df_uni26['Status'].isna()][['Name', 'Status']]

,Name,Status
81,Adelaide University,NaN
261,University of Aberdeen,NaN
309,"City St George’s, University of London",NaN
382,University of Luxembourg,NaN
487,Technische Universität Bergakademie Freiberg,NaN
520,Singapore University of Technology and Design,NaN
530,The Education University of Hong Kong,NaN
573,Isfahan University of Technology,NaN
583,University of Iceland,NaN
697,TUHH Hamburg University of Technology,NaN


In [19]:
# Missing values in 'Status' column have been replaced with the information found online

In [20]:
df_uni26.loc[df_uni26['Name'] == 'Adelaide University', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'University of Aberdeen', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'City St George’s, University of London', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'University of Luxembourg', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'Singapore University of Technology and Design', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'Technische Universität Bergakademie Freiberg', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'Isfahan University of Technology', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'University of Iceland', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'TUHH Hamburg University of Technology', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'Hong Kong Metropolitan University', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'Norwegian University of Life Sciences (UMB)', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'University of Namur', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'Zurich University of Applied Sciences (ZHAW)', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'Addis Ababa University', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'Azerbaijan Technical University', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'University of Stavanger', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'Macao Polytechnic University', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'Osaka Metropolitan University', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'Universidad de Valladolid', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'Universidad Europea de Madrid', 'Status'] = 'Private for Profit',
df_uni26.loc[df_uni26['Name'] == 'Universidad de Córdoba - España', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'University of Cyberjaya', 'Status'] = 'Private not for Profit',
df_uni26.loc[df_uni26['Name'] == 'CEU University', 'Status'] = 'Private not for Profit'
df_uni26.loc[df_uni26['Name'] == 'Universidad de León', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'Université de Bretagne Occidentale (UBO)', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'Université Sorbonne Paris Nord', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'University of Deusto', 'Status'] = 'Private not for Profit',
df_uni26.loc[df_uni26['Name'] == 'University of Ibadan', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'University of Ioannina', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'University of Lagos', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'University of the Algarve', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'Ahmadu Bello University, Zaria', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'Islamic Azad University', 'Status'] = 'Private not for Profit',
df_uni26.loc[df_uni26['Name'] == 'Jahangirnagar University', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'New Mexico State University', 'Status'] = 'Public'
df_uni26.loc[df_uni26['Name'] == 'Technische Universität Kaiserslautern', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'The Education University of Hong Kong', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'Universitat de Lleida', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'Technische Universität Kaiserslautern', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'UCAM Universidad Católica San Antonio de Murcia', 'Status'] = 'Private not for Profit',
df_uni26.loc[df_uni26['Name'] == 'Universidad Nacional de Ingeniería Peru', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'Université de Limoges', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'Universidad Nacional de Ingeniería Peru', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'University of Rajshahi', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'Khulna University', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'Rajshahi University of Engineering and Technology', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'San Francisco State University', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'Tongmyong University', 'Status'] = 'Private not for Profit',
df_uni26.loc[df_uni26['Name'] == 'University of Hawaii at Hilo', 'Status'] = 'Public',
df_uni26.loc[df_uni26['Name'] == 'Kwame Nkrumah University of Science and Technology', 'Status'] = 'Public'

In [21]:
# NaN replacement with info from QS 2025 University Rankings Data Set (available online) 
# St George's, University of London formally merged with City, University of London (August 1, 2024) so 
# we can take the rating of this university - City, University of London = 352

In [22]:
# Replace '-' with NaN in Overall SCORE
df_uni26['Overall SCORE'] = df_uni26['Overall SCORE'].replace('-', np.nan)

# Convert to numeric (so it becomes float64 column)
df_uni26['Overall SCORE'] = pd.to_numeric(df_uni26['Overall SCORE'], errors='coerce')

/var/folders/8c/vl579g1x7f323l1s991t217h0000gn/T/ipykernel_6461/2180070704.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_uni26['Overall SCORE'] = df_uni26['Overall SCORE'].replace('-', np.nan)


In [23]:
df_uni26.loc[df_uni26['Name'] == 'City St George’s, University of London', 'Previous Rank'] = 352

In [24]:
# All NaN values in SCORE variables are kept as NaN to keep the float64 type for these variables 
# to calculate the descriptive statistics 

# 04 Checking duplicates

In [25]:
duplicate_count = df_uni26['Name'].duplicated().sum()
print("Number of duplicate universities:", duplicate_count)

Number of duplicate universities: 0


# 05 Data wrangling. Splitting mixed type values in ranks and scores columns

In [26]:
# Find columns that contain '+' or '=' somewhere
mask = df_uni26.apply(lambda col: col.astype(str).str.contains(r'[+=]', na=False))

# Columns where at least one row has a symbol
cols_with_symbols = mask.any()[mask.any()].index.tolist()

print("Columns with symbols:", cols_with_symbols)

Columns with symbols: ['Rank', 'Previous Rank', 'AR RANK', 'ER RANK', 'FSR RANK', 'CPF RANK', 'IFR RANK', 'ISR RANK', 'ISD RANK', 'IRN RANK', 'EO RANK', 'SUS RANK']


In [27]:
# Columns to split (numbers and symbols)
rank_cols = [
    'AR RANK', 'ER RANK', 'FSR RANK', 'CPF RANK', 'IFR RANK',
    'ISR RANK', 'ISD RANK', 'IRN RANK', 'EO RANK', 'SUS RANK'
]
# Handles '=123', '123=', '701+', '456-', '123', NaN
pattern = re.compile(r'^\s*(=)?\s*(\d+)\s*([+\-=])?\s*$')

def split_num_symbol(val):
    if pd.isna(val):
        return pd.Series([np.nan, np.nan])
    s = str(val).strip()
    m = pattern.match(s)
    if m:
        # number is always group 2; symbol can be leading '=' or trailing + / - / =
        num = float(m.group(2))
        sym = m.group(1) or m.group(3)  # prefer leading '=', else trailing
        return pd.Series([num, sym if sym else np.nan])
    else:
        # value doesn't look like a rank (e.g., 'not ranked')
        return pd.Series([np.nan, np.nan])

# Apply to each selected column and create *_num and *_sym
for col in rank_cols:
    df_uni26[[f'{col}_num', f'{col}_sym']] = df_uni26[col].apply(split_num_symbol)

In [28]:
PLUS_POLICY = 'lower_bound'   # или 'nan'

rank_cols = ['Rank', 'Previous Rank']

_re_band  = re.compile(r'^\s*(\d+)\s*-\s*(\d+)\s*$')   # 701-710, 1001-1200
_re_plus  = re.compile(r'^\s*(\d+)\s*\+\s*$')          # 1401+
_re_tie   = re.compile(r'^\s*=?\s*(\d+)\s*=?\s*$')     # 696, =696, 696=

def to_num_and_sym(val, plus_policy=PLUS_POLICY):
    if pd.isna(val):
        return np.nan, np.nan
    s = str(val).strip()

    m = _re_band.match(s)
    if m:
        lo, hi = float(m.group(1)), float(m.group(2))
        return (lo + hi) / 2.0, np.nan

    m = _re_plus.match(s)
    if m:
        lo = float(m.group(1))
        num = lo if plus_policy == 'lower_bound' else np.nan
        return num, '+'

    m = _re_tie.match(s)
    if m:
        num = float(m.group(1))
        sym = '=' if '=' in s else np.nan
        return num, sym

    return np.nan, np.nan

for col in rank_cols:
    num_col = f'{col}_num'
    sym_col = f'{col}_sym'
    # don't overwrite other rank columns
    if num_col in df_uni26.columns or sym_col in df_uni26.columns:
        continue
    df_uni26[[num_col, sym_col]] = df_uni26[col].apply(
        lambda x: pd.Series(to_num_and_sym(x))
    )


In [29]:
df_uni26.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1503 entries, 0 to 1502
Data columns (total 55 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Index              1503 non-null   int64  
 1   Rank               1503 non-null   object 
 2   Previous Rank      1391 non-null   object 
 3   Name               1503 non-null   object 
 4   Country/Territory  1503 non-null   object 
 5   Region             1503 non-null   object 
 6   Size               1503 non-null   object 
 7   Focus              1503 non-null   object 
 8   Research           1503 non-null   object 
 9   Status             1503 non-null   object 
 10  AR SCORE           1503 non-null   float64
 11  AR RANK            1503 non-null   object 
 12  ER SCORE           1503 non-null   float64
 13  ER RANK            1503 non-null   object 
 14  FSR SCORE          1503 non-null   float64
 15  FSR RANK           1503 non-null   object 
 16  CPF SCORE          1503 

In [30]:
for col in df_uni26.columns.tolist():
      weird = (df_uni26[[col]].map(type) != df_uni26[[col]].iloc[0].apply(type)).any(axis = 1)
      if len (df_uni26[weird]) > 0:
        print (col)

Rank
Previous Rank
AR RANK
ER RANK
FSR RANK
CPF RANK
IFR RANK
ISR RANK
ISD RANK
IRN RANK
EO RANK
SUS RANK
AR RANK_sym
ER RANK_sym
FSR RANK_sym
CPF RANK_sym
IFR RANK_sym
ISR RANK_sym
ISD RANK_sym
IRN RANK_sym
EO RANK_sym
SUS RANK_sym
Rank_sym
Previous Rank_sym


# 05 Countries name changing for consistency

In [31]:
name_fix = {
    'united kingdom': 'United Kingdom of Great Britain and Northern Ireland',
    'venezuela (bolivarian republic of)': 'Venezuela, Bolivarian Republic of',
    'republic of korea': 'Korea, Republic of',  # South Korea

    # Mainland/regions rolled into the sovereign “China” geometry
    'china (mainland)': 'China',
    'hong kong sar, china': 'China', 
    'macao sar, china': 'China',

    # Entities that  named differently or not present
    'puerto rico': 'United States of America',   # aggregate to parent
    'northern cyprus': 'Cyprus',       # aggregate to parent or drop; see options below
}

# 06 Descriptive Statistics Check

In [32]:
df_uni26.describe()

,Index,AR SCORE,ER SCORE,FSR SCORE,CPF SCORE,IFR SCORE,ISR SCORE,ISD SCORE,IRN SCORE,EO SCORE,SUS SCORE,Overall SCORE,AR RANK_num,ER RANK_num,FSR RANK_num,CPF RANK_num,IFR RANK_num,ISR RANK_num,ISD RANK_num,IRN RANK_num,EO RANK_num,SUS RANK_num,Rank_num,Previous Rank_num
count,1503.000000,1503.000000,1503.000000,1503.000000,1503.000000,1416.000000,1466.000000,1466.000000,1501.000000,1503.000000,1479.000000,705.000000,1503.000000,1503.000000,1503.000000,1503.000000,1416.000000,1466.000000,1466.000000,1501.000000,1503.000000,1479.000000,1503.000000,1391.000000
mean,752.000000,25.764937,26.922289,33.979907,30.458150,36.283686,33.352319,34.550546,53.368221,29.952029,51.229750,46.704397,537.759814,537.785096,588.434464,587.314704,575.021893,582.519782,582.620737,587.337775,589.704591,583.885057,746.219228,700.101006
std,434.023041,24.491393,25.495054,28.451886,29.718894,35.236432,32.770330,31.132784,28.907560,29.196361,21.262767,18.841124,222.752827,222.769207,262.055634,261.792546,264.269188,262.963559,263.008923,261.538415,261.644798,262.346838,426.334870,404.843123
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000,25.100000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,376.500000,8.800000,8.500000,10.800000,6.000000,6.600000,5.925000,8.700000,27.600000,6.200000,35.700000,30.900000,376.500000,376.500000,375.500000,375.500000,354.750000,366.250000,366.250000,376.000000,379.500000,370.500000,376.000000,351.000000
50%,752.000000,16.000000,16.400000,23.600000,18.000000,20.100000,19.400000,21.700000,55.800000,17.900000,48.700000,41.500000,701.000000,701.000000,760.000000,751.000000,711.500000,737.500000,737.500000,751.000000,762.000000,737.000000,745.500000,695.500000
75%,1127.500000,32.600000,37.400000,50.550000,49.850000,66.100000,56.700000,55.750000,78.400000,45.900000,66.450000,58.800000,701.000000,701.000000,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,1100.500000,1100.500000
max,1503.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,701.000000,701.000000,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,1401.000000,1401.000000


In [33]:
df_uni26.describe(include=['object'])

,Rank,Previous Rank,Name,Country/Territory,Region,Size,Focus,Research,Status,AR RANK,ER RANK,FSR RANK,CPF RANK,IFR RANK,ISR RANK,ISD RANK,IRN RANK,EO RANK,SUS RANK,AR RANK_sym,ER RANK_sym,FSR RANK_sym,CPF RANK_sym,IFR RANK_sym,ISR RANK_sym,ISD RANK_sym,IRN RANK_sym,EO RANK_sym,SUS RANK_sym,Rank_sym,Previous Rank_sym
count,1503,1391,1503,1503,1503,1503,1503,1503,1503,1503,1503,1503,1503,1416,1466,1466,1501,1503,1479,803,804,716,702,624,670,670,701,715,1330,96,21
unique,400,386,1503,106,5,4,4,4,3,701,700,785,799,791,796,796,800,789,385,1,1,1,1,2,1,1,1,1,2,2,1
top,1201-1400,1001-1200,Youngsan University,United States of America,Asia,L,FC,VH,Public,701+,701+,801+,801+,801+,801+,801+,801+,801+,801+,+,+,+,+,+,+,+,+,+,+,+,+
freq,206,193,1,192,566,677,584,1059,1185,803,804,716,702,622,670,670,701,715,677,803,804,716,702,622,670,670,701,715,677,95,21


In [34]:
df_uni26.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1503 entries, 0 to 1502
Data columns (total 55 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Index              1503 non-null   int64  
 1   Rank               1503 non-null   object 
 2   Previous Rank      1391 non-null   object 
 3   Name               1503 non-null   object 
 4   Country/Territory  1503 non-null   object 
 5   Region             1503 non-null   object 
 6   Size               1503 non-null   object 
 7   Focus              1503 non-null   object 
 8   Research           1503 non-null   object 
 9   Status             1503 non-null   object 
 10  AR SCORE           1503 non-null   float64
 11  AR RANK            1503 non-null   object 
 12  ER SCORE           1503 non-null   float64
 13  ER RANK            1503 non-null   object 
 14  FSR SCORE          1503 non-null   float64
 15  FSR RANK           1503 non-null   object 
 16  CPF SCORE          1503 

# 07 Exporting dataframes

In [35]:
df_uni26.to_excel(
    os.path.join(path, '02_Data', 'Prepared Data', 'QS_Rankings_full_cleaned.xlsx'),
    index=False,
    sheet_name='Cleaned Data'
)

In [36]:
# Exporting another file with only selected columns below
keep_cols = [
    'Rank_num', 'Rank_sym',
    'Previous Rank_num', 'Previous Rank_sym',
    'Name', 'Country/Territory', 'Region', 'Size', 'Focus', 'Research', 'Status', 
    'AR SCORE', 'ER SCORE', 'FSR SCORE', 'CPF SCORE', 'IFR SCORE', 'ISR SCORE',
    'ISD SCORE', 'IRN SCORE', 'EO SCORE', 'SUS SCORE',
    'AR RANK_num', 'AR RANK_sym',
    'ER RANK_num', 'ER RANK_sym',
    'FSR RANK_num', 'FSR RANK_sym',
    'CPF RANK_num', 'CPF RANK_sym',
    'IFR RANK_num', 'IFR RANK_sym',
    'ISR RANK_num', 'ISR RANK_sym',
    'ISD RANK_num', 'ISD RANK_sym',
    'IRN RANK_num', 'IRN RANK_sym',
    'EO RANK_num', 'EO RANK_sym',
    'SUS RANK_num', 'SUS RANK_sym', 'Overall SCORE'
]

# Create new DataFrame
df_ranks = df_uni26[keep_cols].copy()


# save to new Excel
df_ranks.to_excel(
    os.path.join(path, '02_Data', 'Prepared Data', 'QS_Rankings_selected_col_cleaned.xlsx'),
    index=False,
    sheet_name='Cleaned Data'
)